In [1]:
import sys
import os
from datetime import datetime 

import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction import DictVectorizer 
import xgboost as xgb

import mlflow

from misc import init

In [2]:
# initialize configuration
CONFIG = init.init_config()

---User configuration not found. Loading default settings.


In [ ]:
# read data
data = pd.read_parquet(CONFIG["DATA_PATH"])

In [4]:
# setup training context
# get datasets
xtrain, xtest = train_test_split(data, test_size=0.2, random_state=99)

# get targets
ytrain = xtrain[CONFIG["TARGET"]].values
ytest = xtest[CONFIG["TARGET"]].values
xtrain.drop(columns=[CONFIG["TARGET"]], inplace=True)
xtest.drop(columns=[CONFIG["TARGET"]], inplace=True)

# vectorize
dv = DictVectorizer(sparse=False)
train_dict = xtrain.to_dict(orient="records")
test_dict = xtest.to_dict(orient="records")
xtrain = dv.fit_transform(train_dict)
xtest = dv.transform(test_dict)
feature_names = dv.get_feature_names_out().tolist()

# get dmatrix
xtrain = xgb.DMatrix(xtrain, ytrain, feature_names=feature_names)
xtest = xgb.DMatrix(xtest, ytest, feature_names=feature_names)

In [3]:
# # launch mlflow
# mlflow ui --backend-store-uri sqlite:///mlflow/mlflow.db --default-artifact-root mlflow
mlflow.set_tracking_uri("sqlite:///mlflow/mlflow.db")
mlflow.set_experiment(CONFIG["MLFLOW_EXPERIMENT_NAME"])

2024/09/11 09:56:35 INFO mlflow.tracking.fluent: Experiment with name 'political_engagement' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/adi/projects/political-engagement-mlops/mlruns/1', creation_time=1726041395995, experiment_id='1', last_update_time=1726041395995, lifecycle_stage='active', name='political_engagement', tags={}>